In [1]:
from datasets import load_dataset, concatenate_datasets
from transformers import DebertaV2Tokenizer, AutoTokenizer
import ftfy
from tqdm.auto import tqdm

/home/israel/Documents/debertinha/debenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
dataset = load_dataset("dominguesm/wikipedia-ptbr-20230601")

Generating test split: 100%|██████████| 110345/110345 [00:01<00:00, 91604.50 examples/s]


In [4]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")

/home/israel/Documents/debertinha/debenv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
dataset = concatenate_datasets([dataset["train"], dataset["test"]])

In [6]:
dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1103446
})

In [7]:
def batch_iterator(batch_size=10_000):
    for i in tqdm(range(0, len(dataset), batch_size)):
        text = dataset[i : i + batch_size]["text"]
        text = [ftfy.fix_text(t) for t in text]
        yield text

In [8]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=52000)

100%|██████████| 111/111 [41:26<00:00, 22.40s/it]


In [9]:
new_tokenizer.save_pretrained("debertinha-v2-tokenizer")

('debertinha-v2-tokenizer/tokenizer_config.json',
 'debertinha-v2-tokenizer/special_tokens_map.json',
 'debertinha-v2-tokenizer/tokenizer.json')

In [10]:
new_tokenizer.tokenize("Olá, mundo!")

['▁O', 'lá', ',', '▁mundo', '!']

In [11]:
new_tokenizer.tokenize("Eu quero falar sobre um tema tenso hoje")

['▁Eu',
 '▁que',
 'r',
 'o',
 '▁falar',
 '▁sobre',
 '▁um',
 '▁tema',
 '▁',
 'tens',
 'o',
 '▁hoje']

In [15]:
new_tokenizer.sep_token_id

2

In [16]:
new_tokenizer.model_max_length

1000000000000000019884624838656